<a href="https://colab.research.google.com/github/jmcaussade/Arreglo-Incongruencias-Juegos-Jardin/blob/main/Incongruencias_Michel_all_bricks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Código con folder de archivos</h1>


In [ ]:
!pip install xlsxwriter
!pip install matplotlib

In [15]:
import os
import pandas as pd
import xlsxwriter
import unicodedata
import matplotlib
import numpy as np
from glob import glob
import zipfile
import shutil

# Functions
def generate_colors(n):
    """Generate n distinct colors."""
    cmap = matplotlib.colormaps['tab20']
    return [matplotlib.colors.to_hex(cmap(i/n)) for i in range(n)]

def normalizar_texto(texto):
    """Normalize text - convert to lowercase and remove accents."""
    texto = texto.lower()
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode("utf-8")
    return texto

# Define the path of the uploaded zip file
zip_file_path = 'Excel_Bricks.zip'  # Replace with the actual path

# Define the target folder where you want to unzip
target_folder = 'unzipped_folder'  # Replace with your desired target folder path

# Unzip the folder
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(target_folder)

# Use the unzipped folder as input_folder and create the output folder within it
input_folder = os.path.join(target_folder, 'Excel_Bricks')
output_folder = os.path.join(target_folder, 'Excel_Bricks_Clean')
os.makedirs(output_folder, exist_ok=True)

# Get a list of all Excel files in the input folder
excel_files = glob(os.path.join(input_folder, '*.xlsx'))
print(f"excel_files: {excel_files}")
for excel_file in excel_files:
    print(excel_file)
    # Read Excel File
    df = pd.read_excel(excel_file, sheet_name='Hoja1')

    # Preprocessing: Convert all columns to string and strip leading/trailing white spaces
    df = df.applymap(lambda x: str(x).strip() if isinstance(x, str) else str(x))

    # Define columns to exclude
    excluir_columnas = ["<ID>", "SKU - Chile", "Jefe de Línea", 'Descripción Web', '<Name>', "Marca", "Origen", "<Parent ID>"]

    # Generate colors for columns
    initial_col_count = max(len(df.columns) - len(excluir_columnas), 1)
    colores_columnas = generate_colors(initial_col_count)

    # Process DataFrame
    for i, col in enumerate(df.columns):
        if col not in excluir_columnas:
            insert_position = df.columns.get_loc(col) + 1
            col_aparece = f"{col} aparece"

            def calculate_aparece(row):
                value = normalizar_texto(str(row[col]))
                if value in normalizar_texto(str(row['<Name>'])) or value in normalizar_texto(str(row['Descripción Web'])):
                  return 1  # Convert True to integer 1
                else:
                  return 0

            df[col_aparece] = df.apply(calculate_aparece, axis=1)

    # Fill NaN values with 0 in boolean columns
    boolean_columns = [col for col in df.columns if 'aparece' in col]
    df[boolean_columns] = df[boolean_columns].fillna(0).astype(int)

    # Calculate Row-wise Error Percentage
    df['Porcentaje correctitud (Row)'] = df[boolean_columns].apply(lambda row: (row.sum() * 100) / len(row), axis=1)

    # Create a DataFrame to hold the error percentages by column
    error_percentages_col = pd.DataFrame({col: [(df[col].sum() * 100) / len(df[col])] for col in boolean_columns})

    # Save the exported file in the output folder with a unique name
    output_file = os.path.join(output_folder, f'result_{os.path.basename(excel_file)}')
    writer = pd.ExcelWriter(output_file, engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet1', index=False)

    # Formatting and other code for this file...
    # Formatting
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    format_verdadero = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
    format_falso = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})

    for col_num, value in enumerate(df.columns.values):
        if value not in excluir_columnas and 'aparece' not in value:
            color_index = (col_num // 2) % len(colores_columnas)
            cell_format = workbook.add_format({'bg_color': colores_columnas[color_index]})
            worksheet.set_column(col_num, col_num, None, cell_format)
        elif 'aparece' in value:
            worksheet.conditional_format(1, col_num, len(df), col_num,
                                        {'type': 'cell',
                                          'criteria': '=',
                                          'value': 1,  # Use integer 1 instead of True
                                          'format': format_verdadero})
            worksheet.conditional_format(1, col_num, len(df), col_num,
                                        {'type': 'cell',
                                          'criteria': '=',
                                          'value': 0,  # Use integer 0 instead of False
                                          'format': format_falso})

    # Find the index of the first boolean column
    first_boolean_col_index = df.columns.get_loc(boolean_columns[0])

    # Add Error Percentages by Column starting from the first boolean column
    for i, col in enumerate(boolean_columns):
        col_offset = first_boolean_col_index + i  # Calculate the column offset
        for j, value in enumerate(error_percentages_col[col].values):
            worksheet.write(len(df) + 1 + j, col_offset, value if not np.isnan(value) else "NaN")

    # Add "Error Percentage by Attribute" in the first cell of the row after the data ends
    worksheet.write(len(df) + 1, 0, "Porcentaje correctitud por Attribute")

    # Close and save Excel writer for this file
    writer.close()

# End of loop through Excel files
print("All files processed and saved in the output folder.")

# Zip the entire output folder
shutil.make_archive(output_folder, 'zip', output_folder)

print(f"Output folder zipped as {output_folder}.zip")


excel_files: ['unzipped_folder/Excel_Bricks/Televisores 1400.xlsx', 'unzipped_folder/Excel_Bricks/Cemento 2526.xlsx', 'unzipped_folder/Excel_Bricks/Hidrolavadoras.xlsx', 'unzipped_folder/Excel_Bricks/Analisis.xlsx']
unzipped_folder/Excel_Bricks/Televisores 1400.xlsx
unzipped_folder/Excel_Bricks/Cemento 2526.xlsx
unzipped_folder/Excel_Bricks/Hidrolavadoras.xlsx
unzipped_folder/Excel_Bricks/Analisis.xlsx
All files processed and saved in the output folder.
Output folder zipped as unzipped_folder/Excel_Bricks_Clean.zip


<h1>DISPLAY FILE </h1>


In [ ]:
import pandas as pd

# Replace 'your_excel_file.xlsx' with the path to your Excel file
# and 'Sheet1' with the name of the sheet you want to display

file_path = 'unzipped_folder/Excel_Bricks_Clean/result_Analisis.xlsx'
sheet_name = 'Sheet1'
# file_path = 'Analisis.xlsx'
# sheet_name = 'Hoja1'


df = pd.read_excel(file_path, sheet_name=sheet_name)

# Display the DataFrame
# You can use df.head() to display the first few rows
# or df to display the entire DataFrame (be cautious with large DataFrames)
df
